In [ ]:
from datetime import datetime, timezone
from typing import Dict, List, Optional, Tuple
from pyspark.sql import DataFrame
from delta.tables import DeltaTable
from pyspark.sql.types import StructType, StructField, TimestampType, StringType
from pyspark.sql.window import Window
import logging
import pyspark.sql.functions as F

CATALOG = ""
VOLUME_CATALOG = "main"
VOLUME_SCHEMA = "engenharia_dados"
VOLUME_NAME = "aviacao_landing"
LANDING_CSV_BASE_PATH = f"/Volumes/{VOLUME_CATALOG}/{VOLUME_SCHEMA}/{VOLUME_NAME}/aviacao/landing"
BRONZE_SCHEMA = "aviacao_bronze"
SILVER_SCHEMA = "aviacao_silver"
GOLD_SCHEMA = "aviacao_gold"  # Definindo o schema da Gold
META_SCHEMA = "aviacao_meta"
ORIGEM_SISTEMA = "postgres-aviacao"

TABLE_CONFIGS: Dict[str, Dict] = {
    "companhias_aereas": {"schema": "aviacao", "business_key": ["id"]}
}

TABLE_SCHEMAS: Dict[str, StructType] = {}

logger = logging.getLogger("aviacao_gold")
if not logger.handlers:
    handler = logging.StreamHandler()
    formatter = logging.Formatter("%(asctime)s [%(levelname)s] %(name)s - %(message)s")
    handler.setFormatter(formatter)
    logger.addHandler(handler)
logger.setLevel(logging.INFO)

# Funções para criação e verificação de tabelas
def qname(schema: str, table: str) -> str:
    if CATALOG:
        return f"{CATALOG}.{schema}.{table}"
    return f"{schema}.{table}"

def init_schema(schema_name: str) -> None:
    schema_qualified = f"{CATALOG}.{schema_name}" if CATALOG else schema_name
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS {schema_qualified}")

def ensure_schema_exists(schema_name: str) -> None:
    try:
        spark.sql(f"CREATE SCHEMA IF NOT EXISTS {schema_name}")
        logger.info(f"[INFO] Schema {schema_name} criado ou já existente.")
    except Exception as e:
        logger.error(f"[ERROR] Erro ao criar o schema {schema_name}: {str(e)}")

def ensure_gold_table_exists(table_name: str) -> None:
    ensure_schema_exists(GOLD_SCHEMA)
    gold_table = qname(GOLD_SCHEMA, table_name)

    try:
        if not spark.catalog.tableExists(gold_table):
            # Criando tabela Gold para o mart de voos diários (exemplo)
            spark.sql(f"""
                CREATE TABLE {gold_table} (
                    data DATE,
                    cia_sk INT,
                    aeroporto_origem_sk INT,
                    aeroporto_destino_sk INT,
                    qtd_voos_programados INT,
                    qtd_voos_pousados INT,
                    qtd_voos_atrasados INT,
                    qtd_voos_cancelados INT,
                    percentual_atraso DOUBLE,
                    percentual_cancelamento DOUBLE
                )
                USING DELTA
            """)
            logger.info(f"[INFO] Tabela Delta {gold_table} criada com sucesso.")
        else:
            logger.info(f"[INFO] Tabela Delta {gold_table} já existe.")
    except Exception as e:
        logger.error(f"[ERROR] Erro ao criar a tabela Delta {gold_table}: {str(e)}")

# Função para criar o mart de operação de voos
def create_voos_diarios_mart() -> None:
    logger.info("================ INÍCIO CRIAÇÃO MART DE OPERAÇÃO DE VOOS ================")

    # Leitura das tabelas necessárias: fato_voos, dim_aeroportos e dim_companhias_aereas
    fato_voos = spark.table(qname(SILVER_SCHEMA, "fato_voos"))
    dim_aeroportos = spark.table(qname(SILVER_SCHEMA, "dim_aeroportos"))
    dim_companhias_aereas = spark.table(qname(SILVER_SCHEMA, "dim_companhias_aereas"))

    # Agregação dos dados
    df_voos = fato_voos.join(dim_aeroportos, fato_voos.aeroporto_origem_sk == dim_aeroportos.id) \
        .join(dim_aeroportos, fato_voos.aeroporto_destino_sk == dim_aeroportos.id) \
        .join(dim_companhias_aereas, fato_voos.cia_sk == dim_companhias_aereas.id) \
        .groupBy("data", "cia_sk", "aeroporto_origem_sk", "aeroporto_destino_sk") \
        .agg(
        F.count("id").alias("qtd_voos_programados"),
        F.sum(F.when(fato_voos.status == "pousado", 1).otherwise(0)).alias("qtd_voos_pousados"),
        F.sum(F.when(fato_voos.status == "atrasado", 1).otherwise(0)).alias("qtd_voos_atrasados"),
        F.sum(F.when(fato_voos.status == "cancelado", 1).otherwise(0)).alias("qtd_voos_cancelados")
    )

    # Calcular os percentuais
    df_voos = df_voos.withColumn(
        "percentual_atraso", F.col("qtd_voos_atrasados") / F.col("qtd_voos_programados") * 100
    ).withColumn(
        "percentual_cancelamento", F.col("qtd_voos_cancelados") / F.col("qtd_voos_programados") * 100
    )

    # Criar a tabela Gold (mart_voos_diarios)
    ensure_gold_table_exists("mart_voos_diarios")
    df_voos.write.format("delta").mode("overwrite").saveAsTable(qname(GOLD_SCHEMA, "mart_voos_diarios"))

    logger.info("================ FIM CRIAÇÃO MART DE OPERAÇÃO DE VOOS ================")

# Função para criar o mart de receita / comercial
def create_receita_comercial_mart() -> None:
    logger.info("================ INÍCIO CRIAÇÃO MART DE RECEITA COMERCIAL ================")

    # Leitura das tabelas necessárias: fato_bilhetes, fato_reservas, dim_companhias_aereas, dim_voos
    fato_bilhetes = spark.table(qname(SILVER_SCHEMA, "fato_bilhetes"))
    fato_reservas = spark.table(qname(SILVER_SCHEMA, "fato_reservas"))
    dim_companhias_aereas = spark.table(qname(SILVER_SCHEMA, "dim_companhias_aereas"))
    dim_voos = spark.table(qname(SILVER_SCHEMA, "dim_voos"))

    # Agregação dos dados
    df_comercial = fato_bilhetes.join(fato_reservas, fato_bilhetes.reserva_id == fato_reservas.id) \
        .join(dim_companhias_aereas, fato_bilhetes.cia_sk == dim_companhias_aereas.id) \
        .join(dim_voos, fato_bilhetes.voo_sk == dim_voos.id) \
        .groupBy("ano_mes", "cia_sk", "classe_cabine") \
        .agg(
        F.sum("quantidade_bilhetes").alias("qtd_bilhetes"),
        F.sum("valor_bruto").alias("receita_bruta"),
        (F.sum("valor_bruto") / F.sum("quantidade_bilhetes")).alias("receita_media_por_bilhete"),
        (F.sum("quantidade_bilhetes") / F.countDistinct("voo_id")).alias("ocupacao_media")
    )

    # Criar a tabela Gold (mart_receita_cia_mes)
    ensure_gold_table_exists("mart_receita_cia_mes")
    df_comercial.write.format("delta").mode("overwrite").saveAsTable(qname(GOLD_SCHEMA, "mart_receita_cia_mes"))

    logger.info("================ FIM CRIAÇÃO MART DE RECEITA COMERCIAL ================")

# Função para criar o mart de experiência do cliente
def create_experiencia_cliente_mart() -> None:
    logger.info("================ INÍCIO CRIAÇÃO MART DE EXPERIÊNCIA DO CLIENTE ================")

    # Leitura das tabelas necessárias: fato_voos, dim_clientes, fato_reservas
    fato_voos = spark.table(qname(SILVER_SCHEMA, "fato_voos"))
    dim_clientes = spark.table(qname(SILVER_SCHEMA, "dim_clientes"))
    fato_reservas = spark.table(qname(SILVER_SCHEMA, "fato_reservas"))

    # Agregação dos dados
    df_cliente = fato_voos.join(dim_clientes, fato_voos.cliente_id == dim_clientes.id) \
        .join(fato_reservas, fato_voos.reserva_id == fato_reservas.id) \
        .groupBy("cliente_sk", "ano_mes") \
        .agg(
        F.count("id").alias("qtd_voos"),
        F.sum(F.when(fato_voos.status == "atrasado", 1).otherwise(0)).alias("qtd_voos_atrasados"),
        F.sum(F.when(fato_voos.status == "cancelado", 1).otherwise(0)).alias("qtd_voos_cancelados")
    )

    # Calcular o percentual de voos com problemas
    df_cliente = df_cliente.withColumn(
        "percentual_voos_com_problema",
        (F.col("qtd_voos_atrasados") + F.col("qtd_voos_cancelados")) / F.col("qtd_voos") * 100
    )

    # Criar a tabela Gold (mart_atrasos_cliente)
    ensure_gold_table_exists("mart_atrasos_cliente")
    df_cliente.write.format("delta").mode("overwrite").saveAsTable(qname(GOLD_SCHEMA, "mart_atrasos_cliente"))

    logger.info("================ FIM CRIAÇÃO MART DE EXPERIÊNCIA DO CLIENTE ================")

# Função principal para rodar todos os marts
def main() -> None:
    print("Iniciando o processamento da camada Gold...")

    create_voos_diarios_mart()
    create_receita_comercial_mart()
    create_experiencia_cliente_mart()

if __name__ == "__main__":
    main()
